In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv("swiggy.csv")
df1=pd.read_csv("zomato_dataset.csv")

In [3]:
df.head()

,ID,Area,City,Restaurant,Price,Avg ratings,Total ratings,Food type,Address,Delivery time
0,211,Koramangala,Bangalore,Tandoor Hut,300.0,4.4,100,"Biryani,Chinese,North Indian,South Indian",5Th Block,59
1,221,Koramangala,Bangalore,Tunday Kababi,300.0,4.1,100,"Mughlai,Lucknowi",5Th Block,56
2,246,Jogupalya,Bangalore,Kim Lee,650.0,4.4,100,Chinese,Double Road,50
3,248,Indiranagar,Bangalore,New Punjabi Hotel,250.0,3.9,500,"North Indian,Punjabi,Tandoor,Chinese",80 Feet Road,57
4,249,Indiranagar,Bangalore,Nh8,350.0,4.0,50,"Rajasthani,Gujarati,North Indian,Snacks,Desser...",80 Feet Road,63


In [4]:
df1.head()

,Restaurant Name,Rating,Cuisine,Average Price,Average Delivery Time,Safety Measure,Location
0,Campus Bakers,4.3,"Bakery, Fast Food, Pizza, Sandwich, Burger",₹50 for one,36 min,Restaurant partner follows WHO protocol,Agra
1,Mama Chicken Mama Franky House,4,"North Indian, Mughlai, Rolls, Burger, Momos",₹50 for one,22 min,Follows all Max Safety measures to ensure your...,Agra
2,GMB - Gopika Sweets & Restaurant,4.2,"North Indian, South Indian, Chinese, Fast Food...",₹50 for one,27 min,Follows all Max Safety measures to ensure your...,Agra
3,Shree Bankey Bihari Misthan Bhandar,4.2,"Mithai, Street Food, South Indian, Chinese, Ic...",₹50 for one,28 min,Restaurant partner follows WHO protocol,Agra
4,Burger King,4.2,"Burger, Fast Food, Beverages",₹50 for one,26 min,Follows all Max Safety measures to ensure your...,Agra


In [5]:
df1.shape

(44891, 7)

In [9]:
df.shape

(8680, 10)

In [13]:
df1 = df1.iloc[::2].reset_index(drop=True)

In [15]:
df1.shape

(22446, 7)

In [17]:
df.rename(columns={
    'Price': 'Average Price',
    'Avg ratings': 'Rating',
    'Restaurant': 'Restaurant Name',
    'Food type': 'Cuisine',
    'Delivery time': 'Average Delivery Time',
      'City': 'Location'
}, inplace =True)

In [19]:
df = df.drop(columns=['ID', 'Area','Address','Total ratings'])
df1=df1.drop(columns=['Safety Measure'])

In [21]:
df = df[['Restaurant Name', 'Rating', 'Cuisine', 'Average Price','Average Delivery Time','Location']]

In [23]:

# --- Clean "Average Price" ---
df1['Average Price'] = df1['Average Price'].replace(r'\D+', '', regex=True)
df1['Average Price'] = df1['Average Price'].replace('', np.nan)
df1['Average Price'] = df1['Average Price'].astype(float)
df1['Average Price'] = df1['Average Price'].fillna(9999).astype(int)

# --- Clean "Average Delivery Time" ---
df1['Average Delivery Time'] = df1['Average Delivery Time'].replace(r'\D+', '', regex=True)
df1['Average Delivery Time'] = df1['Average Delivery Time'].replace('', np.nan)
df1['Average Delivery Time'] = df1['Average Delivery Time'].astype(float)
df1['Average Delivery Time'] = df1['Average Delivery Time'].fillna(999).astype(int)
df['Average Price'] = df['Average Price'] / 2

# Add a new column 'Platform' and assign 'Swiggy' to the rows in df
df['Platform'] = 'Swiggy'
# Add a new column 'Platform' and assign 'Zomato' to the rows in df1
df1['Platform'] = 'Zomato'

df1['Location'] = df1['Location'].replace('Delhi NCR', 'Delhi')


In [25]:
df.head()

,Restaurant Name,Rating,Cuisine,Average Price,Average Delivery Time,Location,Platform
0,Tandoor Hut,4.4,"Biryani,Chinese,North Indian,South Indian",150.0,59,Bangalore,Swiggy
1,Tunday Kababi,4.1,"Mughlai,Lucknowi",150.0,56,Bangalore,Swiggy
2,Kim Lee,4.4,Chinese,325.0,50,Bangalore,Swiggy
3,New Punjabi Hotel,3.9,"North Indian,Punjabi,Tandoor,Chinese",125.0,57,Bangalore,Swiggy
4,Nh8,4.0,"Rajasthani,Gujarati,North Indian,Snacks,Desser...",175.0,63,Bangalore,Swiggy


In [27]:
df1.head()

,Restaurant Name,Rating,Cuisine,Average Price,Average Delivery Time,Location,Platform
0,Campus Bakers,4.3,"Bakery, Fast Food, Pizza, Sandwich, Burger",50,36,Agra,Zomato
1,GMB - Gopika Sweets & Restaurant,4.2,"North Indian, South Indian, Chinese, Fast Food...",50,27,Agra,Zomato
2,Burger King,4.2,"Burger, Fast Food, Beverages",50,26,Agra,Zomato
3,Goverdhan Bakery,3.9,"Bakery, Fast Food, Desserts",50,29,Agra,Zomato
4,Bikanervala,4.1,"Street Food, North Indian, Chinese, Mithai, So...",50,31,Agra,Zomato


In [29]:

df_merged = pd.concat([df, df1], ignore_index=True)
df_merged['Cuisine'] = df_merged['Cuisine'].str.lower()
df_merged['Location'] = df_merged['Location'].str.lower()
df_merged['Rating'] = pd.to_numeric(df_merged['Rating'], errors='coerce').fillna(0)
df_merged['Average Price'] = pd.to_numeric(df_merged['Average Price'], errors='coerce').fillna(0)
df_merged['Average Delivery Time'] = pd.to_numeric(df_merged['Average Delivery Time'], errors='coerce').fillna(0)

# Define target score (you can adjust the weights based on your preference)
df_merged['Target Score'] = df_merged['Rating'] - 0.1 * (df_merged['Average Price'] / 100) - 0.2 * (df_merged['Average Delivery Time'] / 10)

In [31]:
df_merged.head()

,Restaurant Name,Rating,Cuisine,Average Price,Average Delivery Time,Location,Platform,Target Score
0,Tandoor Hut,4.4,"biryani,chinese,north indian,south indian",150.0,59,bangalore,Swiggy,3.070
1,Tunday Kababi,4.1,"mughlai,lucknowi",150.0,56,bangalore,Swiggy,2.830
2,Kim Lee,4.4,chinese,325.0,50,bangalore,Swiggy,3.075
3,New Punjabi Hotel,3.9,"north indian,punjabi,tandoor,chinese",125.0,57,bangalore,Swiggy,2.635
4,Nh8,4.0,"rajasthani,gujarati,north indian,snacks,desser...",175.0,63,bangalore,Swiggy,2.565


In [33]:
df_merged.shape

(31126, 8)

In [39]:
# Features and target
X = df_merged[['Rating', 'Average Price', 'Average Delivery Time']]
y = df_merged['Target Score']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("RMSE:", root_mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"R² score: {r2}")

RMSE: 0.13703672153395705
R² score: 0.9988156708964118


In [41]:
!pip install gradio --quiet

import gradio as gr
import pandas as pd

# Make sure df_merged and model are already defined before running this cell

def predict_best_restaurant_web(cuisine, location):
    # Clean up the data to be lower case and trimmed
    df_merged['Cuisine'] = df_merged['Cuisine'].str.lower().str.strip()
    df_merged['Location'] = df_merged['Location'].str.lower().str.strip()

    # Clean input
    cuisine = cuisine.strip().lower()
    location = location.strip().lower()

    # Filter Swiggy and Zomato dataframes
    swiggy_restaurants = df_merged[
        df_merged['Cuisine'].str.contains(cuisine, case=False, na=False) &
        df_merged['Location'].str.contains(location, case=False, na=False) &
        (df_merged['Platform'] == 'Swiggy')
    ]

    zomato_restaurants = df_merged[
        df_merged['Cuisine'].str.contains(cuisine, case=False, na=False) &
        df_merged['Location'].str.contains(location, case=False, na=False) &
        (df_merged['Platform'] == 'Zomato')
    ]

    # If no restaurants found
    if swiggy_restaurants.empty and zomato_restaurants.empty:
        return f"❌ No restaurants found for '{cuisine}' cuisine in '{location}'."

    output = ""

    # Swiggy prediction
    swiggy_top_5 = None
    if not swiggy_restaurants.empty:
        swiggy_features = swiggy_restaurants[['Rating', 'Average Price', 'Average Delivery Time']]
        swiggy_restaurants = swiggy_restaurants.copy()
        swiggy_restaurants['Predicted Score'] = model.predict(swiggy_features)
        swiggy_restaurants = swiggy_restaurants[swiggy_restaurants['Predicted Score'] > 0]
        
        swiggy_top_5 = swiggy_restaurants.sort_values(by='Predicted Score', ascending=False).head(5)

        output += f"\n📦 Top {len(swiggy_top_5)} Swiggy Restaurants:\n"
        for idx, row in swiggy_top_5.iterrows():
            output += (
                f"\n🔸 {row['Restaurant Name']}\n"
                f"   Rating: {row['Rating']}/5\n"
                f"   Avg Price: ₹{row['Average Price']}\n"
                f"   Delivery Time: {row['Average Delivery Time']} min\n"
                f"   Predicted Score: {round(row['Predicted Score'], 2)}/5\n"
            )

    # Zomato prediction
    zomato_top_5 = None
    if not zomato_restaurants.empty:
        zomato_features = zomato_restaurants[['Rating', 'Average Price', 'Average Delivery Time']]
        zomato_restaurants = zomato_restaurants.copy()
        zomato_restaurants['Predicted Score'] = model.predict(zomato_features)
        zomato_restaurants = zomato_restaurants[zomato_restaurants['Predicted Score'] > 0]
        
        zomato_top_5 = zomato_restaurants.sort_values(by='Predicted Score', ascending=False).head(5)

        output += f"\n🍽️ Top {len(zomato_top_5)} Zomato Restaurants:\n"
        for idx, row in zomato_top_5.iterrows():
            output += (
                f"\n🔸 {row['Restaurant Name']}\n"
                f"   Rating: {row['Rating']}/5\n"
                f"   Avg Price: ₹{row['Average Price']}\n"
                f"   Delivery Time: {row['Average Delivery Time']} min\n"
                f"   Predicted Score: {round(row['Predicted Score'], 2)}/5\n"
            )

    # Final best overall recommendation
    if swiggy_top_5 is not None and zomato_top_5 is not None:
        all_restaurants = pd.concat([swiggy_top_5, zomato_top_5])
        best_overall = all_restaurants.sort_values(by='Predicted Score', ascending=False).iloc[0]
    elif swiggy_top_5 is not None:
        best_overall = swiggy_top_5.iloc[0]
    else:
        best_overall = zomato_top_5.iloc[0]

    output += (
        f"\n🏆 Final Recommendation:\n"
        f"✅ Best Restaurant: {best_overall['Restaurant Name']} ({best_overall['Platform']})\n"
        f"📈 Predicted Score: {round(best_overall['Predicted Score'], 2)}/5\n"
        f"🌟 Rating: {best_overall['Rating']}/5\n"
    )

    return output.strip()

# Gradio UI
demo = gr.Interface(
    fn=predict_best_restaurant_web,
    inputs=[
        gr.Textbox(label="Enter Cuisine"),
        gr.Textbox(label="Enter Location")
    ],
    outputs="text",
    title="🍴 AI-Powered Restaurant Recommender",
    description="Enter a cuisine and location to get the best restaurant recommendation."
)

# Launch in new browser tab
demo.launch(inbrowser=True,share=True)


* Running on local URL:  http://127.0.0.1:7867

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
